In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model

In [26]:
classes=os.listdir('/content/gdrive/MyDrive/fire_dataset/forest_fire/Training and Validation/')
classes

['nofire', 'fire']

In [27]:
np_clases=np.array(classes)
np_clases

array(['nofire', 'fire'], dtype='<U6')

In [52]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                               validation_split=0.1)

training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/fire_dataset/forest_fire/Training and Validation/',
                                                     target_size = (224,224),
                                                     batch_size = 16,
                                                     class_mode = 'binary',
                                                 subset='training')

Found 1650 images belonging to 2 classes.


In [53]:
val_data = train_datagen.flow_from_directory('/content/gdrive/MyDrive/fire_dataset/forest_fire/Training and Validation/',
                                                     target_size = (224,224),
                                                     batch_size = 16,
                                                     class_mode = 'binary',
                                                    subset='validation'
                                                      )

Found 182 images belonging to 2 classes.


In [64]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_data = test_datagen.flow_from_directory('/content/gdrive/MyDrive/fire_dataset/forest_fire/Testing/',
                                                     target_size = (224,224),
                                                     batch_size = 16,
                                                     class_mode = 'binary'
                                                      )

Found 68 images belonging to 2 classes.


In [55]:
imagesize=[224,224]

In [56]:
ResNet = ResNet50(input_shape=imagesize + [3],weights='imagenet',include_top=False)

In [57]:
for l in ResNet.layers:
  l.trainable = False

In [58]:
x = Flatten()(ResNet.output)

In [59]:
prediction = Dense(1,activation='sigmoid')(x)

model = Model(inputs=ResNet.input, outputs=prediction)

In [60]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
EPOCHS = 50
history = model.fit_generator(training_set,
                    epochs=EPOCHS,
                    validation_data=val_data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
104/104 [==============================] - 178s 2s/step - loss: 0.0851 - accuracy: 0.9661 - val_loss: 0.2361 - val_accuracy: 0.9011
Epoch 2/50
104/104 [==============================] - 178s 2s/step - loss: 0.0769 - accuracy: 0.9770 - val_loss: 0.2351 - val_accuracy: 0.9011
Epoch 3/50
104/104 [==============================] - 178s 2s/step - loss: 0.0948 - accuracy: 0.9630 - val_loss: 0.6675 - val_accuracy: 0.8077
Epoch 4/50
104/104 [==============================] - 179s 2s/step - loss: 0.1353 - accuracy: 0.9461 - val_loss: 0.2260 - val_accuracy: 0.9121
Epoch 5/50
104/104 [==============================] - 179s 2s/step - loss: 0.2054 - accuracy: 0.9255 - val_loss: 0.3306 - val_accuracy: 0.8956
Epoch 6/50
104/104 [==============================] - 179s 2s/step - loss: 0.0832 - accuracy: 0.9667 - val_loss: 0.2392 - val_accuracy: 0.9011
Epoch 7/50
104/104 [==============================] - 178s 2s/step - loss: 0.0953 - accuracy: 0.9612 - val_loss: 0.2313 - val_accuracy: 0.9121

In [73]:
test_accu = model.evaluate_generator(test_data)
test_accu

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.5411400198936462, 0.9558823704719543]

In [74]:
print('The testing accuracy is :',test_accu[1]*100, '%')

The testing accuracy is : 95.58823704719543 %


In [75]:
model.save('/content/gdrive/MyDrive/Saved_Models/fire_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [77]:
model.save_weights('/content/gdrive/MyDrive/Saved_Models/fire')

In [79]:
load_model = tf.keras.models.load_model('/content/gdrive/MyDrive/Saved_Models/fire/fire_model.h5')

In [80]:
load_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________